<a href="https://colab.research.google.com/github/yeb2Binfang/ECE-GY-9163-ML-cyber/blob/main/Lab/LR_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Logistic regression review

## Intro to LR

LR 是很重要的模型，教授在课上也给我们认真地讲过了，这里就把过程以及推论都给详细地写一下

LR是一个很经典的二分类的分类方法，在工业界中，LR也是经常使用的，非常重要，面试也是经常考的，所以我们要去了解怎么推导LR，了解LR的objective function，了解在LR模型下怎么避免overfitting，LR的框架。LR一般都可以作为baseline

在很多场景下都是可以用LR的，比如说
* 贷款违约情况(会/不会)
* 广告点击问题，也就是CTR prediction
* 商品推荐
* 情感分析
* 疾病诊断
* 等等

我们来看一个例子，是一个classification problem，这是一个二分类的问题

现在有这么一个数据

| Age      | Salary | Education      | If Breach of contract |
| :----: | :----: | :----: | :----: |
| 20      | 4000       | undergrad      | Yes     |
| 25   | 5000        | college      | No      |
| 21      | 6000       | undergrad     | No      |
| 25   | 5000       | college     | Yes      |
| 28      | 8000      | undergrad      | No       |
| 27  | 7000        | undergrad      | ?      |

学习输入到输出的映射 $f:X→ Y$

$X$就是我们的属入

$Y$就是我们的输出

这里呢，其实我们也就是想求这么一个概率，也就是$P(Y|X)$。

那我们现在思考一个问题，就是我们可不可以用线性回归来表示$P(Y|X)=w^Tx+b$? 为什么？

当然是不可以了
* 因为$P(Y|X)$是条件概率，它的范围是[0,1]
* 其次就是所有类的概率加起来是1，也就是$∑_y P(Y|X)=1$.
* 而线性回归的范围是(-∞,+∞)

那我们可不可以把线性回归的$w^Tx+b$给映射到[0,1]呢？于是我们就想到了logistic function，也就是
$$
y=\frac{1}{1+e^{-z}}, z=w^Tx+b
$$
它的图像如下图所示

<img src="https://user-images.githubusercontent.com/68700549/147372757-b5acd7b2-7096-4401-b178-bc4bd07e9ce1.png" width="300" height="200" />

于是，我们LR model的decision function就是
$$
P(Y|X)=y=\frac{1}{1+e^{-z}}, z=w^Tx+b
$$

我们现在回到开头提的classification problem，假设模型已经训练好，那么我们可以得到
$$
\begin{gathered}
P(Y=\text{Yes}|(27,7000, \text{undergrad}))=\frac{1}{1+e^{-[\begin{bmatrix}w_1,w_2,w_3\end{bmatrix}\begin{bmatrix}27\\7000\\ \text{undergrad}  \end{bmatrix}+b]}}
\end{gathered}
$$

就是这样的一种形式，然后我们看概率，设置threshold，然后就可以知道是Yes还是No了。

对于二分类问题，我们有
$$
\begin{gathered}
P(y=1|x,w)=\frac{1}{1+e^{-w^Tx+b}}\\P(y=0|x,w)=1-P(y=1|x,w)=\frac{e^{-w^Tx+b}}{1+e^{-w^Tx+b}}
\end{gathered}
$$

上述的两个式子是可以合并成
$$
p(y|x,w)=P(y=1|x,w)^y[1-P(y=1|x,w)]^{1-y}
$$

这里我们思考一个问题，就是LR是线性分类器还是非线性分类器？

答案就是LR是线性分类器。怎么看呢？我们就要去看其决策边界，decision boundary

<img src="https://user-images.githubusercontent.com/68700549/147388134-83139c33-1aca-441d-813b-aa67ea01a843.png" width="427" height="159" />

那么怎么看LR的decision boundary呢？这里呢，有一个假设，假设是二分类的情况，如果是线性分类器的话，就是落在每一边的概率是一样的。于是，对LR来说，就有
$$
\frac{P(y=1|x,w)}{P(y=0|x,w)}=1
$$

于是，我们可以得到$e^{-(w^Tx+b)}=1$，两边加ln，可以得到$lne^{-(w^Tx+b)}=ln1$. 于是可以得到$-(w^Tx+b)=0$, 也就是$w^Tx+b=0$,这也就是LR的decision boundary了，所以，LR的decision boundary是线性的，也就说明了LR是linear classifier。


## LR Objective function

就是我们要去求解$w,b$. LR要优化的函数是最大似然，maximum likelihood。也就是要最大化见到样本的概率。

假设我们拥有数据集$D=\{(x_i,y_i)\}_{i=1}^n,x_i∈ 𝙍^d, y_i∈ \{0,1\}$.

而且，我们已经定义了
$$
p(y|x,w)=P(y=1|x,w)^y[1-P(y=1|x,w)]^{1-y}
$$

我们需要最大化目标函数， MLE=maximum likelihood estimation
$$
\hat{w}_{MLE},\hat{b}_{MLE}=argmax_{w，b} ∏_{i=1}^n p(y_i|x_i,w,b)
$$

所以，在LR中，其实我们是需要考虑所有的样本数据的，我们要去寻找$w,b$使得$∏_{i=1}^n p(y_i|x_i,w,b)$最大化。

接下来我们来看看怎么最大化目标函数,一般来说，我们看到连乘，$∏$,我们就要考虑加上log，log又是单调递增函数，而且会使得连乘运算变得简单
$$
\begin{gathered}
\hat{w}_{MLE},\hat{b}_{MLE}=argmax_{w，b} ∏_{i=1}^n p(y_i|x_i,w,b)\\=argmax_{w,b}log(∏_{i=1}^n p(y_i|x_i,w,b))\\=argmax_{w,b}\sum_{i=1}^n log P(y_i|x_i,w,b)\\=argmin_{w,b}-\sum_{i=1}^n log P(y_i|x_i,w,b)\\=argmin_{w,b}-\sum_{i=1}^n log[P(y=1|x,w)^y[1-P(y=1|x,w)]^{1-y}] \\=argmin_{w,b}-\sum_{i=1}^n ylog P(y=1|x_i, w,b)+(1-y)log[1-P(y=1|x_i,w,b)]
\end{gathered}
$$

这也就是LR 的objective function，一定要记住，也要去理解这是怎么来的

## Gradient descent

我们已经知道了LR 的objective function，接下来就是要知道怎么去优化。在了解怎么优化之前，我们先了解什么是凸函数，上课讲过的，不难理解。我们现在要知道得到的是全局最优解还是局部最优解
<img src="https://user-images.githubusercontent.com/68700549/147388993-10fbc081-c530-4340-8e1f-63cacd520400.png" width="506" height="179" />

典型的非凸就是神经网络

判断一个函数是否是凸函数，就需要看这个函数是否满足
$$
f(λx_1+(1-λ)x_2) ≤ λf(x_1)+(1-λ)f(x_2)
$$

<img src="https://user-images.githubusercontent.com/68700549/147389044-b44655d7-1189-4e01-adfe-f09710bf0e20.png" width="506" height="179" />

常见的convex function有
* 线性函数: $f(x)=a^Tx+b$
* 二次函数: $f(x)=x^TQx+a^Tx+b$, $Q∈ S_+^n$
* least square: $f(x)=||Ax-b||_2^2$
* p-norm: $f(x)=(∑_{i=1}^n|x_i|^p)^{\frac{1}{p}}$, $p ≥ 1$
* log-sum-exp function: $f(x)=log(∑_{i=1}^n e^{x_i}),x\in R^n$






logistic regression本身也是凸函数的

















